In [1]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from simpletransformers.ner import NERModel, NERArgs


In [7]:
data = pd.read_csv("Formatted.csv")

In [6]:
data.head(30)

,Unnamed: 0,sentence_id,word_id,words,labels
0,0,0,0.0,"""",O
1,1,0,0.0,who,O
2,2,0,1.0,may,O
3,3,0,2.0,(,O
4,4,0,2.0,or,O
5,5,0,3.0,may,O
6,6,0,4.0,not,O
7,7,0,5.0,),O
8,8,0,5.0,have,O
9,9,0,6.0,it,O


In [8]:
from sklearn.preprocessing import LabelEncoder
data['sentence_id'] = LabelEncoder().fit_transform(data['sentence_id'])

In [9]:
data.rename(columns = {'word':'words', 'tag':'labels'}, inplace=True)

In [10]:
data['labels'] = data['labels'].str.upper()

In [11]:
train_data = data[['sentence_id', 'words','labels']]

In [39]:
label = data['labels'].unique().tolist()
label

['O', 'B', 'I']

In [13]:
args = NERArgs()
args.num_train_epochs = 2
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size=32
args.eval_batch_size=32

In [14]:
ner_model = NERModel('bert', 'bert-base-cased', labels=label, args = args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [15]:
ner_model.train_model(train_data, acc = accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/189 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 2:   0%|          | 0/189 [00:00<?, ?it/s]

(378, 0.3950328764224809)

In [37]:
result, model_outputs, preds_list = ner_model.eval_model(train_data, output_dir = 'eval/')

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/189 [00:00<?, ?it/s]

In [52]:
preds_list[3]

['B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'B',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [34]:
input = ['username', ' username', ' Without', ' hope', ', we', ' have', ' nothing', '!  The', ' only', ' real', ' hope', ' of', ' ending', ' the', ' epidemic', ' is', ' a', ' cure', '; especially', ' for', ' those', ' who', ' have', ' stage', '3', '-HIV', '. ðŸ’”\n#CommitToACure']
s = ''.join(input)
prediction, model_output = ner_model.predict([s])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'username': 'O'},
  {'username': 'O'},
  {'Without': 'O'},
  {'hope,': 'O'},
  {'we': 'O'},
  {'have': 'O'},
  {'nothing!': 'O'},
  {'The': 'B'},
  {'only': 'I'},
  {'real': 'I'},
  {'hope': 'I'},
  {'of': 'I'},
  {'ending': 'I'},
  {'the': 'I'},
  {'epidemic': 'I'},
  {'is': 'I'},
  {'a': 'I'},
  {'cure;': 'I'},
  {'especially': 'I'},
  {'for': 'I'},
  {'those': 'I'},
  {'who': 'I'},
  {'have': 'I'},
  {'stage3-HIV.': 'I'},
  {'ðŸ’”': 'O'},
  {'#CommitToACure': 'O'}]]